SBB-APP, this notebook will gather all the differents code, Data Importation, Manipulation, Cleaning & Exploratory about data related to the Train Network in Switzerland. The final aim is to develop an interactive Dashboard with Bokeh. 

Please note that on the 17th of October 18, we are still struggling to get more data to complete the Dashboard. 

Besides, I will gather in a single notebook all the different Script.

I will be glad to get your commments,


In [36]:
"""PART1 - Data Gathering
We start with gathering all the different data that will be needed

As a I find a map one of the most interactive and efficient way to communicate your data
I went on http://www.diva-gis.org/gdata to download the SHP of Switzerland, and we will use GeoPandas to be able to read this SHP file

Then, fortunately, we can Access the Open Data Portal of SBB (Swissrail company) providing different datasets to get the information we would like.
After looking at the different proposed dataset, we will be working mainly with the wollowing : passagierfrequenz.csv, including information about passengers and Swiss Stop.
"""

# -*- coding: utf-8 -*-

import pandas as pd #DataManipulation
import geopandas as gpd #Convert SHP file into readable DataFrame
import os #To get path to Folders and File
from bokeh.models import ColumnDataSource, GeoJSONDataSource 
#ColumnDataSource -> Way for bokeh to read your data
from bokeh.plotting import figure, show, output_notebook
#GeoJSONDataSource -> Fortunately, Bokeh developed a way to read Geometry Information through GeoJSONDataSource

%matplotlib inline 
#To be able to plot within the Notebook 






"""PART1 - A - Data Importation & Cleaning > our future DataFrame data_passengers"""

data_passengers = pd.read_csv('passagierfrequenz.csv', sep=';',
                              engine='python', encoding = 'utf-8',
                              parse_dates=['Bezugsjahr'])

data_passengers['Year'] = data_passengers['Bezugsjahr'].astype(str).str[0:4].astype(float) #create a new column including only the year.
#There is no information about 2015 or the other months of 2014-2016. Therefore we will keep split it into two columns. But first, let's clean the DF and drop Useless/incomplete information

data_passengers = data_passengers.set_index('Bezugsjahr').drop(['Bemerkung', 'lod'], axis= 1).dropna(subset = ['geopos']) #Drop unrelevant columns and NaN in Geopos
data_passengers[['DTV', 'DMW']] = data_passengers[['DTV', 'DMW']].fillna(0) #Let's convert NaN into float for DTV
#DTV and DMW correspond to Number of Passengers : DTV(Monday to Friday), DMW(Monday to Sunday)


"""After a quick view of the DF, we see that we could plot an HBAR to represent the number of passengers for 2014 & 2016 = > data_passengers_on_change [on_change '''for CallBack''' (See Bokeh) & Update DataSource"""
#data_passengers_chart, we take off geopos to make the file lighter
data_passengers = data_passengers.drop(['geopos'], axis= 1)

#We create new columns 2016 & 2014 and drop Year.
data_passengers_2016 = data_passengers[data_passengers['Year'] == 2016 ].sort_values('Bahnhof_Haltestelle') #DF of Data Passengers for the Year 2016
data_passengers_2014 = data_passengers[data_passengers['Year'] == 2014 ].sort_values('Bahnhof_Haltestelle') #DF of Data Passengers for the Year 2014

data_passengers_2016 = data_passengers_2016.reset_index() # To avoid duplicating index
data_passengers_2014 = data_passengers_2014.reset_index() # To avoid duplicating index

data_passengers_on_change = data_passengers_2016.drop('Bezugsjahr', axis= 1) #Become Useless with the column Year

data_passengers_on_change['DTV_2016'] = data_passengers_on_change['DTV'] #Get the DTV for 2016
data_passengers_on_change['DMW_2016'] = data_passengers_on_change['DMW'] #Get the DMW for 2016

data_passengers_on_change['Year_2014'] = data_passengers_2014['Year']
data_passengers_on_change['DTV_2014'] = data_passengers_2014['DTV']
data_passengers_on_change['DMW_2014'] = data_passengers_2014['DMW']

data_passengers_on_change = data_passengers_on_change.drop(['DTV', 'DMW'], axis = 1) #Drop now the duplicates 

data_passengers_on_change = data_passengers_on_change.rename(columns={'Year': 'Year_2016'})

#We create the ColumnDataSource for HBar Figure - DTW & DMW for 2014 + 2016
source_pass = ColumnDataSource(data_passengers_on_change)

data_passengers_on_change.head()

""" Data Top 10-20-30-50-100 I find it interesting to do this to get different interation with the chat.""" 
"""We create these DataFrame to update ColumnDataSource according to selection on TAB 2"""

top_AZ = data_passengers_on_change.sort_values('Bahnhof_Haltestelle') #DF Sorted by Alphateic order

top_2016 = data_passengers_on_change.drop(['Year_2014', 'DTV_2014', 'DMW_2014'], axis = 1) #DF for 2016
top_2014 = data_passengers_on_change.drop(['Year_2016', 'DTV_2016', 'DMW_2016'], axis = 1) #DF for 2014
top_10_DMW16 = data_passengers_on_change.sort_values('DMW_2016', ascending = False).iloc[0:10] #DF2016 -top 10
top_10_DMW16_rev = data_passengers_on_change.sort_values('DMW_2016', ascending = True).iloc[0:10]
top_20_DMW16 = data_passengers_on_change.sort_values('DMW_2016', ascending = False).iloc[0:20] #DF2016 -top 20
top_20_DMW16_rev = data_passengers_on_change.sort_values('DMW_2016', ascending = True).iloc[0:20]
top_50_DMW16 = data_passengers_on_change.sort_values('DMW_2016', ascending = False).iloc[0:50] #DF2016 -top 50
top_100_DMW16 = data_passengers_on_change.sort_values('DMW_2016', ascending = False).iloc[0:10] #DF2016 -top 100
top_500_DMW16 = data_passengers_on_change.sort_values('DMW_2016', ascending = False).iloc[0:500] #DF2016 -top 500

top_10_DMW14 = data_passengers_on_change.sort_values('DMW_2014', ascending = False).iloc[0:10] #DF2014 - Top10
top_20_DMW14 = data_passengers_on_change.sort_values('DMW_2014', ascending = False).iloc[0:20]
top_50_DMW14 = data_passengers_on_change.sort_values('DMW_2014', ascending = False).iloc[0:50]
top_100_DMW14 = data_passengers_on_change.sort_values('DMW_2014', ascending = False).iloc[0:10]
top_500_DMW14 = data_passengers_on_change.sort_values('DMW_2014', ascending = False).iloc[0:500]

data_40000 =  data_passengers_on_change[data_passengers_on_change['DTV_2016'] < 40000] #DF >40 000 - Focus on Small Stations




"""PART1 - B - Data Importation & Cleaning - Map, our Tab N°3"""
"""MAP - Part 1 : Importation GeoData /Data Geolocalisation"""

#Shapefile in DataFrame via GeoPandas to draw the SwissMap
cwd = os.getcwd()
temp_cwd = cwd + '\\geo_data\\'
gdf0 = gpd.read_file(temp_cwd + 'gadm36_CHE_0.shp') #GDF0 is SwissMap only
gdf1 = gpd.read_file(temp_cwd + 'gadm36_CHE_1.shp') #GDF1 is SwissMap + Canton (Administatrives Areas)
gdf0_crs = gdf0.to_crs({'init': 'epsg:3857'})
gdf1_crs = gdf1.to_crs({'init': 'epsg:3857'}) #Important to match the CRS to Flat Map (See on Google)

#Shapefile in DataFrame via GeoPandas to draw the Stations - Provided on OPENPORTAL SBB
gdf_loc_station = gpd.read_file('lorastation.shp', encoding = 'utf-8')
gdf_loc_station = gdf_loc_station.drop(20).reset_index(drop= True)
gdf_loc_station_crs = gdf_loc_station.to_crs({'init': 'epsg:3857'})

#Shapefile in DataFrame via GeoPandas to draw the Train Lines - Provided on OPENPORTAL SBB
gdf_loc_line = gpd.read_file('linie-mit-betriebspunkten.shp', encoding = 'utf-8')
gdf_loc_line = gdf_loc_line.sort_values(['linie', 'km']) #Tri pour dessiner les arrêts les uns après les autres.
gdf_loc_line_crs = gdf_loc_line.to_crs({'init': 'epsg:3857'}) #Conversion au format CRS 3857



""" MAP - Part 2 : We need a unique DF to plot the TrainLines"""

gdf_loc_line = gdf_loc_line.drop(['lod', 'what3word_e','what3word_i', 'what3word_f', 'what3word_d'], axis = 1) #drop irrelevant columns
#We need to create a single dataframe for each line.

def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

list_df = []
list_x_coord = []
list_y_coord = []

list_lines = gdf_loc_line.linie.unique().tolist() #Get the list of the lines

# Calculate x coordinates to avoid connections between two differents lines
gdf_loc_line_crs['x'] = gdf_loc_line_crs.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
# Calculate y coordinates pour éviter les connections entre deux lifgnes différents
gdf_loc_line_crs['y'] = gdf_loc_line_crs.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

for i, linie in enumerate(list_lines) :
    df_line = gdf_loc_line_crs[gdf_loc_line_crs['linie'] == linie]
    a = df_line.x.tolist()
    list_x_coord.append(a)
    b = df_line.y.tolist()
    list_y_coord.append(b)

gdf_map_crs = gdf_loc_line_crs[['linie', 'linienname']].drop_duplicates().reset_index(drop = True)
serie_x_coord = pd.Series(list_x_coord)
serie_y_coord = pd.Series(list_y_coord)
gdf_map_crs['x_coord'] = serie_x_coord
gdf_map_crs['y_coord'] = serie_y_coord
source_map_crs = ColumnDataSource(gdf_map_crs)

list_lines = gdf_loc_line.linie.unique().tolist()
list_namelines = gdf_loc_line.linienname.unique().tolist()
gdf0_crs = gdf0.to_crs({'init': 'epsg:3857'})
source_geo_crs = GeoJSONDataSource(geojson=gdf0_crs.to_json())
source_geo_station = GeoJSONDataSource(geojson=gdf_loc_station.to_json())
source_geo_station_crs = GeoJSONDataSource(geojson=gdf_loc_station_crs.to_json())
source_geo_line = GeoJSONDataSource(geojson=gdf_loc_line.to_json())
source_geo_line_crs = GeoJSONDataSource(geojson=gdf_loc_line_crs.to_json())

data_passengers_on_change.head()

,Code,Bahnhof_Haltestelle,Eigner,Year_2016,DTV_2016,DMW_2016,Year_2014,DTV_2014,DMW_2014
0,AD,Aadorf,SBB,2016.0,1700.0,2000.0,2014.0,1700.0,2000.0
1,AA,Aarau,SBB,2016.0,38000.0,44400.0,2014.0,35900.0,42000.0
2,ABO,Aarburg-Oftringen,SBB,2016.0,2300.0,2700.0,2014.0,2000.0,2500.0
3,AAT,Aathal,SBB,2016.0,690.0,750.0,2014.0,770.0,860.0
4,AE,Aesch,SBB,2016.0,1900.0,2200.0,2014.0,2100.0,2400.0


Now that we have our Data imported and Clean let's start working on the Dashboard.
I think that the most ideal, is to start our first onglet as TableColumn to give an idea to the users with which data they are dealing with.


In [44]:
from bokeh.io import curdoc
from bokeh.models import Tabs, Panel
from bokeh.models.widgets import TableColumn, DataTable
from bokeh.layouts import row, column
from bokeh.models.widgets.tables import DateFormatter

output_notebook()

#Table Lines Localisation
gdf_loc_line_table = gdf_loc_line.drop('geometry', axis= 1)
source_table = ColumnDataSource(gdf_loc_line_table.sort_values(['bezeichnung']))

columns = [
        TableColumn(field = 'bezeichnung', title = 'Stopname', width = 140),
        TableColumn(field = 'linie', title = 'Lines n°', width = 80),
        TableColumn(field = 'linienname', title = 'Line Name', width = 200),
        TableColumn(field = 'km', title = 'Number of Km from Starting point', width = 100)
    ]

data_table_loc = DataTable(source=source_table, columns=columns, height = 500, width = 1000)

#Table Passengers
source_table_passengers = ColumnDataSource(data_passengers)

columns_passengers = [
    TableColumn(field = 'Bezugsjahr', title = 'Date', formatter = DateFormatter(format = "%Y")),
    TableColumn(field = 'Code', title = 'Code'),
    TableColumn(field = 'Bahnhof_Haltestelle', title = 'Stop'),
    TableColumn(field = 'DTV', title = 'DTV'),
    TableColumn(field = 'DMW', title = 'DMW'),
    TableColumn(field = 'Eigner', title = 'Eigner'),
                     ]

data_table_passengers = DataTable(source = source_table_passengers, columns = columns_passengers, height = 500, width = 1000)


#Create the layout, column
layout_table = column(data_table_loc, data_table_passengers)

show(layout_table)

onglet_1 = Panel(child = layout_table, title="Data") #For later

Loading BokehJS ...

In [47]:
import os
from bokeh.io import curdoc
from bokeh.models import HoverTool, Select, Span, LabelSet, Label
from bokeh.models.widgets import TableColumn, DataTable, Slider
from bokeh.plotting import figure
from bokeh.transform import dodge
from bokeh.core.properties import value
from bokeh.layouts import widgetbox



#load data.py
from data import *

from bokeh.layouts import row, column

source_pass = ColumnDataSource(data_passengers_on_change)

# Create a new plot: plot
fig_hbar = figure(x_range = (0, max(data_passengers_on_change['DMW_2016']) + 50000),
           y_range = data_passengers_on_change['Bahnhof_Haltestelle'].sort_values(ascending = False),
           plot_height = 10000, plot_width=1000,
           title = 'Passenger per Station')

# Add a line to the plot
p_hbar1 = fig_hbar.hbar(y=dodge('Bahnhof_Haltestelle', +0.30, range = fig_hbar.y_range) , right = 'DTV_2014',
              source = source_pass,  
              height = 0.20,
              color="#718dbf", legend=value("DTV 2014"))

p_hbar3 = fig_hbar.hbar(y=dodge('Bahnhof_Haltestelle', +0.10, range = fig_hbar.y_range) , right = 'DMW_2014',
              source = source_pass,  
              height = 0.20,
              color="lightblue", legend=value("DMW 2014"))

p_hbar2 = fig_hbar.hbar(y=dodge('Bahnhof_Haltestelle', -0.10, range = fig_hbar.y_range) , right = 'DTV_2016',
              source = source_pass,  
              height = 0.20,
              color="#e84d60", legend=value("DTV 2016"))

p_hbar4 = fig_hbar.hbar(y=dodge('Bahnhof_Haltestelle', -0.30, range = fig_hbar.y_range) , right = 'DMW_2016',
              source = source_pass,  
              height = 0.20,
              color="pink", legend=value("DMW 2016"))




#Add HoverTool
fig_hbar.add_tools(HoverTool(mode='hline', renderers = [p_hbar1], tooltips = [('DTV_2014', '@DTV_2014'),
                                                   ('Station', '@Bahnhof_Haltestelle')]))

fig_hbar.add_tools(HoverTool(mode='hline', renderers = [p_hbar3], tooltips = [('DMW_2014', '@DMW_2014'),
                                                   ('Station', '@Bahnhof_Haltestelle')]))

fig_hbar.add_tools(HoverTool(mode='hline', renderers = [p_hbar2], tooltips = [('DTV_2016', '@DTV_2016'),
                                                   ('Station', '@Bahnhof_Haltestelle')]))

fig_hbar.add_tools(HoverTool(mode='hline', renderers = [p_hbar4], tooltips = [('DMV_2016', '@DMW_2016'),
                                                   ('Station', '@Bahnhof_Haltestelle')]))
#Add Span
p_hbar5 = Span(location=10000,
               dimension='height', line_color='green',
               line_dash='dashed', line_width=2)

fig_hbar.add_layout(p_hbar5)


#Add Slider to Span (fig_hbar5)
slider = Slider(start=0, end=500000, value=p_hbar5.location, step=5000, title="Span Bar")

def callback_span(attr, new, old) :
    p_hbar5.location = slider.value
slider.on_change('value', callback_span)

#Remove inconvenient formatter / Layout
fig_hbar.xaxis[0].formatter.use_scientific = False
fig_hbar.ygrid.grid_line_color = None
fig_hbar.legend.location = "top_right"
fig_hbar.legend.orientation = "horizontal"
fig_hbar.xaxis.axis_label = 'Number of Passengers'
fig_hbar.yaxis.axis_label = 'City Name'


#Add Callback - 1st Menu
menu = Select(options=['Toutes', 'Top 10', 'Top 20', '<= 40000 Passengers'], value='Toutes', title='Nombres de Villes')

def callback(attr, old, new):
    if menu.value == 'Toutes' : 
        source_pass.data = ColumnDataSource(data_passengers_on_change).data
        fig_hbar.height = 10000
        fig_hbar.width = 3000
        fig_hbar.x_range.start = 0
        fig_hbar.x_range.end = (max(data_passengers_on_change['DMW_2016']) + 50000)
        fig_hbar.y_range.factors = data_passengers_on_change['Bahnhof_Haltestelle'].sort_values(ascending = False).tolist()
        
    elif menu.value == 'Top 10':
        source_pass.data = ColumnDataSource(top_10_DMW16).data
        fig_hbar.y_range.factors = top_10_DMW16['Bahnhof_Haltestelle'].sort_values(ascending = False).tolist()
        fig_hbar.x_range.start = 0
        fig_hbar.x_range.end = (max(data_passengers_on_change['DMW_2016']) + 50000)
        fig_hbar.height = 800
        fig_hbar.width = 1000
        
    elif menu.value == 'Top 20' : 
        source_pass.data = ColumnDataSource(top_20_DMW16).data
        fig_hbar.y_range.factors = top_20_DMW16['Bahnhof_Haltestelle'].sort_values(ascending = False).tolist()
        fig_hbar.height = 800
        fig_hbar.width = 1000
        fig_hbar.x_range.start = 0
        fig_hbar.x_range.end = (max(data_passengers_on_change['DMW_2016']) + 50000)
        
    elif menu.value == '<= 40000 Passengers' :
        source_pass.data = ColumnDataSource(data_40000).data
        fig_hbar.y_range.factors = data_40000['Bahnhof_Haltestelle'].sort_values(ascending = False).tolist()
        fig_hbar.height = 9000
        fig_hbar.width = 2000
        fig_hbar.x_range.start = 0
        fig_hbar.x_range.end = (max(data_40000['DMW_2016']) + 2000)
        

menu.on_change('value', callback)

# 2nd Menu 
menu_order = Select(options=['A => Z', 'Z => A'], value = 'A => Z', title = "Order")

def callback_order (attr, old, new) : 
    if menu_order.value =='A => Z' : 
        list_y_range = list(sorted(fig_hbar.y_range.factors, reverse = True))
        fig_hbar.y_range.factors = list_y_range
    elif menu_order.value == 'Z => A' :
        list_y_range = list(sorted(fig_hbar.y_range.factors))
        fig_hbar.y_range.factors = list_y_range       

    
    
menu_order.on_change('value', callback_order)

#3rd Menu

menu_year = Select(options=['2014', '2016', '2014/16'], value='2014/16', title='Year(s)')

def callback_year (attr, old, new) : 
    if menu_year.value == '2014/16' : 
        if menu_dtv.value == 'DTV + DMV' : 
            p_hbar1.visible = True
            p_hbar2.visible = True
            p_hbar3.visible = True
            p_hbar4.visible = True
        elif menu_dtv.value == 'DTV' :
            p_hbar1.visible = True
            p_hbar2.visible = True
            p_hbar3.visible = False
            p_hbar4.visible = False
        elif menu_dtv.value == 'DMW' :
            p_hbar1.visible = False
            p_hbar2.visible = False
            p_hbar3.visible = True
            p_hbar4.visible = True
            
    elif menu_year.value == '2014' :
        if menu_dtv.value == 'DTV + DMV' : 
            p_hbar1.visible = True
            p_hbar2.visible = False
            p_hbar3.visible = True
            p_hbar4.visible = False
        elif menu_dtv.value == 'DTV' :
            p_hbar1.visible = True
            p_hbar2.visible = False
            p_hbar3.visible = False
            p_hbar4.visible = False
        elif menu_dtv.value == 'DMW' :
            p_hbar1.visible = False
            p_hbar2.visible = False
            p_hbar3.visible = True
            p_hbar4.visible = False
            
    elif menu_year.value == '2016' :
        if menu_dtv.value == 'DTV + DMV' : 
            p_hbar1.visible = False
            p_hbar2.visible = True
            p_hbar3.visible = False
            p_hbar4.visible = True
        elif menu_dtv.value == 'DTV' :
            p_hbar1.visible = False
            p_hbar2.visible = True
            p_hbar3.visible = False
            p_hbar4.visible = False
        elif menu_dtv.value == 'DMW' :
            p_hbar1.visible = False
            p_hbar2.visible = False
            p_hbar3.visible = False
            p_hbar4.visible = True
        

menu_year.on_change('value', callback_year)

row1 = row(menu, menu_order, menu_year, slider)
layout_hbar = column(row1, fig_hbar)

show(layout_hbar)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html

